In [27]:
using JLD2, Flux, CairoMakie, JLD2, Random
include("DenseNTk.jl"); include("FastNTKMethods.jl")


kernel (generic function with 3 methods)

In [29]:
Random.seed!(123)

x = zeros(2,10)
x[1,:] = randn(10)
x[2,:] = randn(10)
f = x->exp.(sin.(x[1,:]*pi)+cos.(x[2,:]*pi))
y = f(x)

act = relu; N1=10_000; InDim = size(x)[1]; OutDim = 1
model = Chain(DenseNTK(InDim=>N1,activation),DenseNTK(N1=>OutDim))|>f64
params_0 = deepcopy(Flux.params(model))
Loss(model,x,y) = Flux.mse(y,model(x))
data = [(x,y)]

K = kernel(model,x,false); eig = eigen(K).values; lr_opt1 = 1/eig[end]; lr_opt2 = 1/eig[end] * 10

10×10 Matrix{Float64}:
 2.01192   0.345179   2.03881    0.301484   …  0.448665  1.1513     0.953648
 0.345179  1.66707    1.21193    1.35652       1.74642   1.43876    0.672351
 2.03881   1.21193    4.91215    0.0785283     1.78312   2.93552   -0.135534
 0.301484  1.35652    0.0785283  1.46423       1.25358   0.75094    1.09012
 0.667712  0.93796   -0.15233    1.26211       0.772915  0.429737   1.3097
 0.912913  0.899608   0.472083   1.0707     …  0.820265  0.701916   1.15881
 0.887961  1.20385    1.37577    1.02053       1.27383   1.2792     0.804278
 0.448665  1.74642    1.78312    1.25358       1.91221   1.74607    0.491012
 1.1513    1.43876    2.93552    0.75094       1.74607   2.12766    0.302199
 0.953648  0.672351  -0.135534   1.09012       0.491012  0.302199   1.40578

In [ ]:
L_opt1 = []
@showprogress for epoch = 1:n_epoch
    Flux.train!(Loss,model,data,Descent(lr_opt1))
    push!(L_opt1,Loss(model),x,y)